# Kaggle TPS 2021 10 Data Prep

The October dataset is particularly large so if you run this in colab, you'll need High RAM

## PIP Installs

In [1]:
import os
from google.colab import output
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
working_dir = 'MyDrive/kaggle/202110'
drive_dir = '/content/drive/'
model_dir = 'models'
output_dir = 'output'
data_dir = 'data'
from google.colab import drive
drive.mount(drive_dir, force_remount=True)  # Do this once only

mounted_drive = f'{drive_dir}/{working_dir}' #''
os.chdir(mounted_drive)

Mounted at /content/drive/


If you want to directly download the kaggle files to your Google Colab environment

From https://www.analyticsvidhya.com/blog/2021/05/10-colab-tips-and-hacks-for-efficient-use-of-it/

https://github.com/Kaggle/kaggle-api

You can pull the datasets directly from Kaggle
1.  Go to your account page on kaggle.com and scroll to API to get your token
2.  Download the token 
3.  Copy it to Google Drive - most people put it in the .kaggle directory
4.  Set enviornment variable KAGGLE_CONFIG_DIR
5.  ```
! pip install kaggle
kaggle competitions download -c tabular-playground-series-oct-2021```
6.  You may need to add logic to move the files where you want them

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/.kaggle"

In [6]:
!pip install kaggle
!kaggle competitions download -c tabular-playground-series-oct-2021

 98% 431M/438M [00:07<00:00, 52.4MB/s]
100% 438M/438M [00:08<00:00, 57.3MB/s]
 89% 1.00M/1.12M [00:00<00:00, 2.96MB/s]
100% 1.12M/1.12M [00:00<00:00, 2.84MB/s]
100% 875M/877M [00:20<00:00, 69.1MB/s]
100% 877M/877M [00:20<00:00, 44.7MB/s]


In [10]:

!mv test.csv.zip data/test.csv.zip
!mv train.csv.zip data/train.csv.zip
!cd data; unzip train.csv.zip; unzip test.csv.zip
!ls data

mv: cannot stat 'test.csv.zip': No such file or directory
mv: cannot stat 'train.csv.zip': No such file or directory
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
artifacts  data  models  sample_submission.csv.zip


In [3]:
df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')


In [4]:
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

# Massaging Data

These are taken from Kaggle notebooks (see commented code)


1.   Memory optimization 
     * scans each column and based on the numeric range, sets the type to the one that requires the least memory
     * improves performance because the datasets are very large
2.   Imputation
     * uses sklearn's iterative imputer which approximates the missing values based on "nearby" values.
     * improves scores
     * note that it takes a while, so for this notebook, the massaged data is being saved locally
3.   additional columns are added to indicate the number of rows missing data and the missing feature columns
4.  Another option seen in other competition notebooks: replace nulls with mean

In [6]:
RUN_MASSAGE = True

In [7]:
# https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021
# Derived from the original script https://www.kaggle.com/gemartin/load-data-reduce-memory-usage 
# by Guillaume Martin

# This checks the range of values in each column and resets the type 

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [8]:
df = reduce_mem_usage(df)
test_df = reduce_mem_usage(test_df)

Mem. usage decreased to 963.21 Mb (56.0% reduction)
Mem. usage decreased to 481.13 Mb (55.9% reduction)


In [9]:
if RUN_MASSAGE:
  X = df.drop(columns = ['id', 'target'])
  test_X = test_df.drop(columns = ['id'])

### Iterative Imputation (requires high RAM)

In [10]:
#https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021
# Reference: https://www.kaggle.com/hsuchialun/tps-lightgbm-kfold
if RUN_MASSAGE:
    feats = list(X.columns[:-1])

    X['n_row_missing'] = X[feats].isna().sum(axis=1)
    test_X['n_row_missing'] = test_X[feats].isna().sum(axis=1)

    X['row_std'] = X[feats].std(axis=1)
    test_X['row_std'] = test_X[feats].std(axis=1)

    feats += ['n_row_missing', 'row_std']


In [11]:
# MULTIVARIATE ITERATIVE IMPUTATION

if RUN_MASSAGE:
    
    imp = IterativeImputer(max_iter=10, random_state=0)
    imp.fit(X[feats].sample(n=10_000))

    X[feats] = imp.transform(X[feats])
    test_X[feats] = imp.transform(test_X[feats])

In [12]:
if RUN_MASSAGE:
    pd.DataFrame(X).to_csv('./data/reduced_mem_imputed.csv', index=False)
    pd.DataFrame(test_X).to_csv('./data/reduced_mem_imputed_test.csv', index=False)

In [ ]:
X = pd.read_csv('./reduced_mem_imputed.csv')
test_X = pd.read_csv('./reduced_mem_imputed_test.csv') # this is the Kaggle test dataset

# Alternative Imputing

Below ONLY outlines what I saw in some other notebooks

In [13]:
# https://www.kaggle.com/hsuchialun/tps-lightgbm-kfold

FEATURES = list(df.columns[:-1])
TARGET = df.columns[-1]  # for this dataset, target is the last column


In [14]:
df[FEATURES] = df[FEATURES].fillna(df[FEATURES].mean()) # have also seen mode
test_df[FEATURES] = test_df[FEATURES].fillna(test_df[FEATURES].mean())


In [15]:
df.to_csv('mean_imputed_train.csv', index=False)
test_df.to_csv('mean_imputed_test.csv', index=False)